In [2]:
# construct all the important rhea files (esp. rhea2formula)
from bioservices import Rhea
r = Rhea()
df = r.search("")
df.head()

INFO    [bioservices.Rhea:363]:  Initialising Rhea service (REST)


,Reaction identifier,Equation,ChEBI name,ChEBI identifier,EC number,Enzymes,PubMed,Cross-reference (EcoCyc),Cross-reference (MetaCyc),Cross-reference (KEGG),Cross-reference (Reactome)
0,RHEA:10000,H2O + pentanamide = NH4(+) + pentanoate,H2O;pentanamide;NH4(+);pentanoate,CHEBI:15377;CHEBI:16459;CHEBI:28938;CHEBI:31011,EC:3.5.1.50,0,NaN,NaN,MetaCyc:PENTANAMIDASE-RXN,KEGG:R02938,NaN
1,RHEA:10004,benzyl isothiocyanate = benzyl thiocyanate,benzyl isothiocyanate;benzyl thiocyanate,CHEBI:17484;CHEBI:16017,EC:5.99.1.1,0,13997458,NaN,MetaCyc:THIOCYANATE-ISOMERASE-RXN,KEGG:R04010,NaN
2,RHEA:10008,[protein]-dithiol + a hydroperoxide = [protein...,L-cysteine residue;a hydroperoxide;L-cystine r...,CHEBI:29950;CHEBI:35924;CHEBI:50058;CHEBI:3087...,NaN,8,12033427;9587003;12517450;10751410,NaN,NaN,NaN,NaN
3,RHEA:10012,(R)-6-hydroxynicotine + H2O + O2 = 6-hydroxyps...,(R)-6-hydroxynicotine;H2O;O2;6-hydroxypseudoox...,CHEBI:58413;CHEBI:15377;CHEBI:15379;CHEBI:5868...,EC:1.5.3.6,1,16095622;2680607,NaN,MetaCyc:R-6-HYDROXYNICOTINE-OXIDASE-RXN,KEGG:R07170,NaN
4,RHEA:10016,H2O + O-sinapoylcholine = (E)-sinapate + choli...,H2O;O-sinapoylcholine;(E)-sinapate;choline;H(+),CHEBI:15377;CHEBI:16353;CHEBI:30023;CHEBI:1535...,EC:3.1.1.49,1,18036206,NaN,MetaCyc:SINAPINE-ESTERASE-RXN,KEGG:R02381,NaN


In [65]:
import compress_pickle
import itertools
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle

## Map Rhea to proper directions (master or bi)

In [4]:
new_direc = pd.read_csv("rhea-directions_18jan2023.tsv", delimiter='\t', keep_default_na=False)
# print(old_direc.shape)

(13651, 4)
(15453, 4)


In [5]:
all2rhea_bi = dict()
all2rhea_master = dict()

for idx in new_direc.index:
  one_row = new_direc.loc[idx, :]
  # updating all2bi
  for direction_type in one_row.index:
    all2rhea_bi['RHEA:'+str(one_row[direction_type])] = 'RHEA:'+str(one_row['RHEA_ID_BI'])
    all2rhea_master['RHEA:'+str(one_row[direction_type])] = 'RHEA:'+str(one_row['RHEA_ID_MASTER'])

In [124]:
# compress_pickle.dump(all2rhea_bi, 'rhea_all2bi_18jan2023.lzma',
#                      compression="lzma", set_default_extension=False)
# compress_pickle.dump(all2rhea_master, 'rhea_all2master_18jan2023.lzma',
#                      compression="lzma", set_default_extension=False)

In [26]:
## Just for testing
# with open('rhea_all2bi_18jan2023.lzma', 'rb') as f:
#   rhea2bi = compress_pickle.load(f)
# with open('rhea_all2master_18jan2023.lzma', 'rb') as f:
#   rhea2master = compress_pickle.load(f)

## Next, get Rhea2CHEBI & Rhea2Formula

In [21]:
CHEBI_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/chebi'
with open(os.path.join(CHEBI_DIR, 'chebi_shortened_formula_30apr2022.pickle'), 'rb') as f:
  chebi2sformula = pickle.load(f)
with open(os.path.join(CHEBI_DIR, 'chebi_second2prime_29apr2022.pickle'), 'rb') as f:
  chebi2prime = pickle.load(f)

In [22]:
chebi2prime['CHEBI:16651']

'CHEBI:16651'

In [80]:
mrhea2chebi = dict()
mrhea2sformula = dict()
for idx in df.index:
  one_row = df.loc[idx, :]
  one_mrhea = all2rhea_master[one_row['Reaction identifier']]
  chebi_ids = one_row['ChEBI identifier'].split(';')
  ## TODO: NEED TO MAP TO PRIMARY TERMS FIRST; 
  chebi_primes = [chebi2prime[val] for val in chebi_ids if val in chebi2prime.keys()]
  # only store unique elements
  sforms = list(set([chebi2sformula[val] for val in chebi_primes if val in chebi2sformula.keys()]))
  # map once again, as long as elemennts exist;
  if len(sforms)>0:
    mrhea2chebi[one_mrhea] = chebi_primes
    mrhea2sformula[one_mrhea] = sforms

In [83]:
# compress_pickle.dump(mrhea2chebi, 'mrhea2chebi_prime_18jan2023.lzma',
#                      compression="lzma", set_default_extension=False)
# compress_pickle.dump(mrhea2sformula, 'mrhea2sformula_18jan2023.lzma',
#                      compression="lzma", set_default_extension=False)

## Now, Create REF-MAT (or REF-DAT) 

In [84]:
# 1. collect all formulas (columns)
formulas = []
for one_k in mrhea2sformula.keys():
  one_list_formula = mrhea2sformula[one_k]
  formulas.append(one_list_formula)
all_formulas = list(set(itertools.chain(*formulas)))

# set index as the list of Rhea terms
ref_mat = pd.DataFrame(0, index=list(mrhea2sformula.keys()), columns=all_formulas)

for one_k in mrhea2sformula.keys():
  one_list_formula = mrhea2sformula[one_k]
  ref_mat.loc[one_k, one_list_formula] = 1

In [85]:
ref_mat.shape

(15451, 4003)

In [101]:
# ref_dat is a decomposed version of ref_mat, to reduce size
# do it by columns (smaller number than rhea reactions)
nonzero_vals = []
for cidx in range(len(ref_mat.columns)):
  one_col = ref_mat.iloc[:, cidx]
  nonzero_vals.append((cidx, list(one_col.to_numpy().nonzero()[0])))
ref_dat = (all_formulas, list(mrhea2sformula.keys()), nonzero_vals)

In [102]:
# testing if data works well
# first of list is list of columns
cols = ref_dat[0]
# second, list of indices
inds = ref_dat[1]
# third, list of index (column, [non-zero rows])
ref_mat_pairs = ref_dat[2]
REF_MAT = pd.DataFrame(0, index=inds, columns=cols)
for val in ref_mat_pairs:
  REF_MAT.iloc[val[1], val[0]] = 1

In [107]:
np.sum(np.sum(ref_mat == REF_MAT)) == ref_mat.shape[0] * ref_mat.shape[1]

True

In [116]:
# compress_pickle.dump(ref_dat, 'data2ref_mat_18jan2023.lzma',
#                      compression="lzma", set_default_extension=False)

## Next, EC2Rhea(Master) and KEGG2Rhea(Master) for taking out existing annotations

In [44]:
rhea2ec = pd.read_csv("rhea2ec_18jan2023.tsv", delimiter='\t', keep_default_na=False)
rhea2kegg = pd.read_csv("rhea2kegg_reaction_18jan2023.tsv", delimiter='\t', keep_default_na=False)
rhea2go = pd.read_csv("rhea2go_18jan2023.tsv", delimiter='\t', keep_default_na=False)

In [122]:
kegg2mrhea = dict()
for idx in rhea2kegg.index:
  one_row = rhea2kegg.loc[idx, :]
  kegg_str = 'KEGG:'+one_row['ID']
  if kegg_str in kegg2mrhea.keys():
    kegg2mrhea[kegg_str].append('RHEA:'+str(one_row['MASTER_ID']))
  else:
    kegg2mrhea[kegg_str] = ['RHEA:'+str(one_row['MASTER_ID'])]
print(len(kegg2mrhea))
#
ec2mrhea = dict()
for idx in rhea2ec.index:
  one_row = rhea2ec.loc[idx, :]
  ec_str = 'EC:'+one_row['ID']
  if ec_str in ec2mrhea.keys():
    ec2mrhea[ec_str].append('RHEA:'+str(one_row['MASTER_ID']))
  else:
    ec2mrhea[ec_str] = ['RHEA:'+str(one_row['MASTER_ID'])]
print(len(ec2mrhea))
#
go2mrhea = dict()
for idx in rhea2go.index:
  one_row = rhea2go.loc[idx, :]
  go_str = one_row['ID']
  if go_str in go2mrhea.keys():
    go2mrhea[go_str].append('RHEA:'+str(one_row['MASTER_ID']))
  else:
    go2mrhea[go_str] = ['RHEA:'+str(one_row['MASTER_ID'])]
print(len(go2mrhea))

6637
5964
4354


In [123]:
compress_pickle.dump(kegg2mrhea, 'kegg2mrhea_18jan2023.lzma',
                     compression="lzma", set_default_extension=False)
compress_pickle.dump(ec2mrhea, 'ec2mrhea_18jan2023.lzma',
                     compression="lzma", set_default_extension=False)
compress_pickle.dump(go2mrhea, 'go2mrhea_18jan2023.lzma',
                     compression="lzma", set_default_extension=False)